In [13]:
import os
import sys

import numpy as np
import pandas as pd
import polars as pl
import sqlite3
import torch

In [22]:
sys.path.append('../..')

from endure.data.io import Reader
from endure.lsm.cost import EndureCost
from endure.lsm.types import Policy, System, LSMDesign, LSMBounds, Workload
from endure.lsm.solver import ClassicSolver, KLSMSolver

Settings to force LaTeX style font within matplotlib plots

In [29]:
config = Reader.read_config('../../endure.toml')
bounds = LSMBounds(**config["lsm"]["bounds"])
bounds.size_ratio_range = [2, 21]
cf = EndureCost(max_levels=bounds.max_considered_levels)
solver = ClassicSolver(bounds)
ksolver = KLSMSolver(bounds)

In [30]:
WL_COLUMNS = ['empty_reads', 'non_empty_reads', 'range_queries', 'writes']
SYS_COLUMNS = ['entry_size', 'selectivity', 'entries_per_page', 'num_elmement', 'bits_per_elem_max', 'read_write_asym']

In [31]:
connection = sqlite3.connect("../../axe_data.db")
env_table = pl.read_database("SELECT * FROM environments;", connection)
run_table = pl.read_database("SELECT * FROM tunings;", connection)

In [32]:
cursor = connection.cursor()
cursor.execute("DROP TABLE IF EXISTS monkey_tunings;")
cursor.close()
connection.commit()

Create the monkey tunings table, this will hold tunings and their associated cost

In [33]:
cursor = connection.cursor()
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS monkey_tunings (
        env_id INTEGER PRIMARY KEY AUTOINCREMENT,
        bits_per_elem REAL,
        size_ratio REAL,
        is_leveling INTEGER,
        cost REAL,
        kcost REAL,
        FOREIGN KEY (env_id) REFERENCES workloads(env_id)
    );
    """
)
cursor.close()
connection.commit()

Finding all of the tunings, here we limit everything to be leveling or tiering

In [35]:
cursor = connection.cursor()
environment_ids = cursor.execute("SELECT env_id FROM environments").fetchall()
for (env_id,) in environment_ids:
    env = env_table.filter(pl.col('env_id') == env_id)
    data = run_table.filter(pl.col('env_id') == env_id)
    wl = Workload(*env[WL_COLUMNS].rows()[0])
    system = System(*env[SYS_COLUMNS].rows()[0])
    design, _ = solver.get_nominal_design(system, wl.z0, wl.z1, wl.q, wl.w)
    design.T = np.around(design.T)
    kdesign, _ = ksolver.get_nominal_design(system, wl.z0, wl.z1, wl.q, wl.w)
    cost = cf.calc_cost(design, system, wl.z0, wl.z1, wl.q, wl.w)
    kcost = cf.calc_cost(kdesign, system, wl.z0, wl.z1, wl.q, wl.w)
    cursor.execute(
        """
        INSERT OR REPLACE INTO monkey_tunings (
            env_id,
            bits_per_elem,
            size_ratio,
            is_leveling,
            cost,
            kcost)
        VALUES (?, ?, ?, ?, ?, ?)
        """,
        (env_id, design.h, design.T, 1 if design.policy == Policy.Leveling else 0, cost, kcost)
    )
cursor.close()
connection.commit()

In [36]:
monkey_table = pl.read_database("SELECT * FROM monkey_tunings;", connection)
monkey_table

env_id,bits_per_elem,size_ratio,is_leveling,cost,kcost
i64,f64,f64,i64,f64,f64
1,3.211367,6.0,1,27.830134,27.820038
2,8.778876,6.0,1,1.157511,1.155549
3,6.72313,7.0,1,2.08895,2.08806
4,1.0,20.0,1,99.338312,99.338312
5,3.311264,20.0,0,2.682739,2.672113
…,…,…,…,…,…
11,1.0,6.0,1,53.681294,53.653926
12,4.526605,20.0,1,34.305867,34.305867
13,8.006566,9.0,0,2.378837,2.250784


In [21]:
# connection.close()